In [13]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import time
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

#config
#from config import sql_password

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# URL of page to be scraped
url = 'https://www.indeed.com/jobs?q=data%20analyst&l=austin'
browser.visit(url)

In [4]:
title_list=[]
company_list=[]
location_list=[]
link_description_list=[]

for x in range(1, 75):
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.find_all('div', class_="jobsearch-SerpJobCard")
    
    for result in results:
        title=result.a['title']
        title_list.append(title)
        company=result.find('span', class_='company').text.strip()
        company_list.append(company)
        location=result.find(class_='location').text
        location_list.append(location)
        link_description=result.a['href']
        link_description_list.append(link_description)

In [5]:
description_url_list = ['http://indeed.com' + url for url in link_description_list]

In [6]:
description_text_list=[]
URL=[]

for urls in description_url_list:
    URL.append(urls)
    browser.visit(urls)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    description_results=soup.find_all('div', class_="jobsearch-JobComponent-description")
    for description in description_results:
        actual_description=description.text
    description_text_list.append(actual_description)
#         print(urls)
#         print(actual_description)
#         print('------------------------')

In [7]:
jobs_df = pd.DataFrame({
    "Job Title": title_list,
    "Company": company_list,
    "Location": location_list,
    "URL": URL
})

In [23]:
jobs_df

,Job Title,Company,Location,URL
0,Healthcare Data Analyst,TML MultiState IEBP,"Austin, TX 78754",http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
1,Data Analyst,Oxford Global Resources,"Austin, TX",http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
2,Data Analyst (Entry Level),Express Employment Professionals,"Bee Cave, TX",http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
3,Senior Data Analyst,ShipStation,"Austin, TX 78756 (Rosedale area)",http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
4,TRACS Data Analyst 06-19,Housing Authority of The City of Austin,"Austin, TX 78704 (South Lamar-South Congress a...",http://indeed.com/rc/clk?jk=c54e5625ad2987c2&f...
5,"Analyst, Team Member Insights",Whole Foods Market,"Austin, TX 78703",http://indeed.com/rc/clk?jk=97fc95e36028d58c&f...
6,Data Operations Analyst,ForeFlight,"Austin, TX 78701 (Downtown area)",http://indeed.com/rc/clk?jk=2069233c26f280cd&f...
7,Data Analyst,360training.com,"Austin, TX",http://indeed.com/company/360training.com/jobs...
8,Data Analyst,BidPrime,"Austin, TX 78741 (Pleasant Valley area)",http://indeed.com/company/BidPrime/jobs/Data-A...
9,C3 Presents- Data Analyst,Live Nation,"Austin, TX 78702 (Rosewood area)",http://indeed.com/rc/clk?jk=ed50b7e3369b1541&f...


In [9]:
description_df = pd.DataFrame({
    "Description": description_text_list,
    "URL": URL
})

In [10]:
description_df.head()

,Description,URL
0,SUMMARY: TML Multistate IEBP is pleased to ann...,http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
1,ContractJob Description:The role is for a data...,http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
2,$18 - $20 an hourOur client in SW Austin (near...,http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
3,ContractShipStation is an Austin-based softwar...,http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
4,$18.50 an hourJOB NOTICE\n DATE POSTED: Februa...,http://indeed.com/rc/clk?jk=c54e5625ad2987c2&f...


In [14]:
# Connect to local database
rds_connection_string = "root:KB@127.0.0.1"
engine = create_engine(f'mysql://{rds_connection_string}')

In [15]:
engine.execute("CREATE DATABASE jobs_db;")

In [16]:
engine.execute("USE jobs_db;")

In [17]:
# executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
# browser = Browser('chrome', **executable_path, headless=False)

In [18]:
# # Connect to local database
# rds_connection_string = "root:KB@127.0.0.1"
# engine = create_engine(f'mysql://{rds_connection_string}')
# conn = engine.connect()

### Use pandas to load DataFrame into database

In [19]:
jobs_df.to_sql(name='job_postings', con=engine, if_exists='replace', index=False)

In [20]:
description_df.to_sql(name='description', con=engine, if_exists='replace', index=False)

### Query from SQL

In [21]:
pd.read_sql_query('select * from job_postings', con=engine).head()

,Job Title,Company,Location,URL
0,Healthcare Data Analyst,TML MultiState IEBP,"Austin, TX 78754",http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
1,Data Analyst,Oxford Global Resources,"Austin, TX",http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
2,Data Analyst (Entry Level),Express Employment Professionals,"Bee Cave, TX",http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
3,Senior Data Analyst,ShipStation,"Austin, TX 78756 (Rosedale area)",http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
4,TRACS Data Analyst 06-19,Housing Authority of The City of Austin,"Austin, TX 78704 (South Lamar-South Congress a...",http://indeed.com/rc/clk?jk=c54e5625ad2987c2&f...


In [22]:
pd.read_sql_query('select * from description', con=engine).head()

,Description,URL
0,SUMMARY: TML Multistate IEBP is pleased to ann...,http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
1,ContractJob Description:The role is for a data...,http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
2,$18 - $20 an hourOur client in SW Austin (near...,http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
3,ContractShipStation is an Austin-based softwar...,http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
4,$18.50 an hourJOB NOTICE\n DATE POSTED: Februa...,http://indeed.com/rc/clk?jk=c54e5625ad2987c2&f...
